In this notebook data are divided in weeks, to be able to compute weekly sentiment.

******OPEN DATAFRAME WITH POLARS******

In [1]:
#Parallelism
import polars as pl

#Import modules
import matplotlib.pyplot as plt
%matplotlib inline

#handling data
import pandas as pd
import numpy as np
import json
import networkx as nx
from datetime import datetime

import random

In [ ]:
#load data
data_sub=pl.read_ndjson(r"\\193.205.65.209\Data\Marta_Baratto\Reddit\submissions\RS_2016-04")
data_com=pl.read_ndjson(r"\\193.205.65.209\Data\Marta_Baratto\Reddit\comments\RC_2016-04")

In [3]:
#select columns
df_sub=data_sub.select(["id","author","title","selftext","name","score","num_comments","subreddit","subreddit_id","from","from_id","created_utc"])
df_com=data_com.select(["id","author","body","parent_id","link_id","subreddit","subreddit_id","created_utc"])

# #expression to call columns
# df_pl['selftext']

#Dataframe length
print('la lunghezza del dataframe di submissions iniziale è ' + str(len(df_sub)))
print('la lunghezza del dataframe di comments iniziale è ' + str(len(df_com)))

la lunghezza del dataframe di submissions iniziale è 6922510
la lunghezza del dataframe di comments iniziale è 64271256


In [4]:
df_com.head()

id,author,body,parent_id,link_id,subreddit,subreddit_id,created_utc
str,str,str,str,str,str,str,i64
"""d1l58qp""","""Lets-MAGA-now""","""Stop talking t…","""t1_d1l4guz""","""t3_4cow6o""","""The_Donald""","""t5_38unr""",1459468800
"""d1l58qq""","""[deleted]""","""[removed]""","""t3_4cqk00""","""t3_4cqk00""","""gaming""","""t5_2qh03""",1459468800
"""d1l58qr""","""TR_Ollington""","""I found out af…","""t1_d1l4gra""","""t3_4cr0a8""","""DestinyTheGame…","""t5_2vq0w""",1459468800
"""d1l58qs""","""Diamondzacks""","""A Long Way to …","""t3_4cr5x8""","""t3_4cr5x8""","""printSF""","""t5_2rzjk""",1459468800
"""d1l58qt""","""Sir_Wylde""","""Splarnst...I b…","""t1_d1l3a1m""","""t3_4cqw8y""","""vexillology""","""t5_2rygj""",1459468800


In [5]:
name_list=[]
for i in range(len(df_com)):
    name="t1_" + df_com["id"][i]
    name_list.append(name)

df_com=df_com.with_columns(pl.Series(name="name", values=name_list))

In [6]:
df_com.head()

id,author,body,parent_id,link_id,subreddit,subreddit_id,created_utc,name
str,str,str,str,str,str,str,i64,str
"""d1l58qp""","""Lets-MAGA-now""","""Stop talking t…","""t1_d1l4guz""","""t3_4cow6o""","""The_Donald""","""t5_38unr""",1459468800,"""t1_d1l58qp"""
"""d1l58qq""","""[deleted]""","""[removed]""","""t3_4cqk00""","""t3_4cqk00""","""gaming""","""t5_2qh03""",1459468800,"""t1_d1l58qq"""
"""d1l58qr""","""TR_Ollington""","""I found out af…","""t1_d1l4gra""","""t3_4cr0a8""","""DestinyTheGame…","""t5_2vq0w""",1459468800,"""t1_d1l58qr"""
"""d1l58qs""","""Diamondzacks""","""A Long Way to …","""t3_4cr5x8""","""t3_4cr5x8""","""printSF""","""t5_2rzjk""",1459468800,"""t1_d1l58qs"""
"""d1l58qt""","""Sir_Wylde""","""Splarnst...I b…","""t1_d1l3a1m""","""t3_4cqw8y""","""vexillology""","""t5_2rygj""",1459468800,"""t1_d1l58qt"""


In [7]:
from datetime import datetime

def convert(text):
    return datetime.fromtimestamp(int(text))

datetime_creation_com=data_com['created_utc'].map_elements(convert)
df_com=df_com.with_columns(datetime_creation_com)

datetime_creation_sub=data_sub['created_utc'].map_elements(convert)
df_sub=df_sub.with_columns(datetime_creation_sub)

***FILTRO SUBREDDIT***

In [9]:
#data_com.columns
#link_id: è l'id della submission a cui si fa riferimento
#parent_id: è l'id della submission o del comment a cui sta rispondendo
#name: è l'id completo del comment
#author: è l'autore

In [9]:
#Se ci sono testi vuoti (es. ci sono delle immagini) usiamo i titoli come testi
body=[]
for i in range(len(df_sub)):
    if(df_sub['selftext'][i]==""):
        body.append(df_sub['title'][i])
    else:
        body.append(df_sub['selftext'][i])

df_sub=df_sub.with_columns(pl.Series(name="body", values=body))

In [10]:
df_sub=df_sub.select(["id","author","body","name","subreddit","subreddit_id","created_utc"])
df_com=df_com.select(["id","author","body","name","subreddit","subreddit_id","created_utc","link_id"])

In [11]:
# df_com=df_com.with_columns(data_com.select(["link_id"]))

In [12]:
if (df_sub.columns==df_com.columns):
    print('YES')

In [13]:
# df_sub.write_ndjson(r"C:\Tempor\Reddit\submissions\RS_2015-01_body.json")

In [14]:
df_sub_wsb=df_sub.filter((df_sub['subreddit']=="wallstreetbets")|(df_sub['subreddit']=="finance")|
                         (df_sub['subreddit']=="investing")|(df_sub['subreddit']=="Frugal")|
                         (df_sub['subreddit']=="SecurityAnalysis")|(df_sub['subreddit']=="RobinHood")|
                         (df_sub['subreddit']=="InvestmentClub")|(df_sub['subreddit']=="Forex")|
                         (df_sub['subreddit']=="options")|(df_sub['subreddit']=="CryptoCurrency")|
                         (df_sub['subreddit']=="StockMarket")|(df_sub['subreddit']=="economy")|
                         (df_sub['subreddit']=="Economics")|(df_sub['subreddit']=="GlobalMarkets")|
                         (df_sub['subreddit']=="dividends")|(df_sub['subreddit']=="Daytrading")|
                         (df_sub['subreddit']=="algotrading")|(df_sub['subreddit']=="pennystocks")|
                         (df_sub['subreddit']=="ValueInvesting")|(df_sub['subreddit']=="Bogleheads")|
                         (df_sub['subreddit']=="RobinHoodPennyStocks")|(df_sub['subreddit']=="RealDayTrading")|
                         (df_sub['subreddit']=="investingforbeginners")|(df_sub['subreddit']=="business")|
                         (df_sub['subreddit']=="Flipping")|(df_sub['subreddit']=="Entrepreneur")|
                         (df_sub['subreddit']=="wallstreet")|(df_sub['subreddit']=="wallstreetdd")|
                         (df_sub['subreddit']=="MiddleClassFinance")|(df_sub['subreddit']=="beermoney")|
                         (df_sub['subreddit']=="DropShipping101")|(df_sub['subreddit']=="smallbusiness")|
                         (df_sub['subreddit']=="churning")|(df_sub['subreddit']=="giftcardexchange")|
                         (df_sub['subreddit']=="personalfinance")|(df_sub['subreddit']=="FinancialPlanning")|
                         (df_sub['subreddit']=="CRedit")|(df_sub['subreddit']=="povertyfinance")|
                         (df_sub['subreddit']=="financialindipendence")|(df_sub['subreddit']=="leanfire")|
                         (df_sub['subreddit']=="flatFIRE")|(df_sub['subreddit']=="StudentLoans"))
                         

df_com_wsb=df_com.filter((df_com['subreddit']=="wallstreetbets")|(df_com['subreddit']=="finance")|
                         (df_com['subreddit']=="investing")|(df_com['subreddit']=="Frugal")|
                         (df_com['subreddit']=="SecurityAnalysis")|(df_com['subreddit']=="RobinHood")|
                         (df_com['subreddit']=="InvestmentClub")|(df_com['subreddit']=="Forex")|
                         (df_com['subreddit']=="options")|(df_com['subreddit']=="CryptoCurrency")|
                         (df_com['subreddit']=="StockMarket")|(df_com['subreddit']=="economy")|
                         (df_com['subreddit']=="Economics")|(df_com['subreddit']=="GlobalMarkets")|
                         (df_com['subreddit']=="dividends")|(df_com['subreddit']=="Daytrading")|
                         (df_com['subreddit']=="algotrading")|(df_com['subreddit']=="pennystocks")|
                         (df_com['subreddit']=="ValueInvesting")|(df_com['subreddit']=="Bogleheads")|
                         (df_com['subreddit']=="RobinHoodPennyStocks")|(df_com['subreddit']=="RealDayTrading")|
                         (df_com['subreddit']=="investingforbeginners")|(df_com['subreddit']=="business")|
                         (df_com['subreddit']=="Flipping")|(df_com['subreddit']=="Entrepreneur")|
                         (df_com['subreddit']=="wallstreet")|(df_com['subreddit']=="wallstreetdd")|
                         (df_com['subreddit']=="MiddleClassFinance")|(df_com['subreddit']=="beermoney")|
                         (df_com['subreddit']=="DropShipping101")|(df_com['subreddit']=="smallbusiness")|
                         (df_com['subreddit']=="churning")|(df_com['subreddit']=="giftcardexchange")|
                         (df_com['subreddit']=="personalfinance")|(df_com['subreddit']=="FinancialPlanning")|
                         (df_com['subreddit']=="CRedit")|(df_com['subreddit']=="povertyfinance")|
                         (df_com['subreddit']=="financialindipendence")|(df_com['subreddit']=="leanfire")|
                         (df_com['subreddit']=="flatFIRE")|(df_com['subreddit']=="StudentLoans"))

print('la lunghezza del dataframe di submissions filtrato è ' + str(len(df_sub_wsb)))
print('la lunghezza del dataframe di comments filtrato è ' + str(len(df_com_wsb)))

la lunghezza del dataframe di submissions filtrato è 58268
la lunghezza del dataframe di comments filtrato è 384556


In [ ]:
# df_sub_wsb.write_ndjson(r'C:\Tempor\reddit\dati_subreddit\sub_2016_4.json')
# df_com_wsb.write_ndjson(r'C:\Tempor\reddit\dati_subreddit\com_2016_4.json')

In [18]:
# df_sub_wsb=pl.read_ndjson(r'C:\Tempor\reddit\dati_subreddit\sub_2015_5.json')
# df_com_wsb=pl.read_ndjson(r'C:\Tempor\reddit\dati_subreddit\com_2015_5.json')

In [19]:
# df_sub_wsb

In [20]:
text=df_sub_wsb['body']
text_2=df_com_wsb['body']
# #Apple
# A=np.array(text.str.contains('|'.join(['iphone','apple tv','apple ipad','apple id','apple mac','apple ipod',
#                                        'apple macbook','apple itunes','apple trailers','apple support','apple mini',
#                                        'macbook pro','apple ipod touch']), literal=False))
# #Apple Inc.(qui metto quelle doppie: apple store, apple stock, apple computer)
# C=np.array(text.str.contains('|'.join(['apple inc','apple history','apple inc, history','apple company','apple stock',
#                                        'apple inc. stock','apple jobs', 'apple, inc.','apple store','apple computer',
#                                        'steve jobs','apple information','apple mission statement','history of apple inc']), literal=False))


# RES=np.array(text.str.contains('|'.join(['Apple','APPLE','AAPL','aapl'])))
# RES2=np.array(text_2.str.contains('|'.join(['Apple','APPLE','AAPL','aapl'])))
# # RES=np.logical_or(A,C)
# df_sub_wsb_apple=df_sub_wsb.filter(RES)
# print('la lunghezza del dataframe di submissions filtrato è ' + str(len(df_sub_wsb_apple)))

# # authors=df_sub_wsb_apple['name'].to_list()
# # Bool_vector=np.array(df_com_wsb['link_id'].str.contains('|'.join(authors)))
# # df_com_wsb_apple=df_com_wsb.filter(Bool_vector)
# df_com_wsb_apple=df_com_wsb.filter(RES2)
# print('la lunghezza del dataframe di comments filtrato è ' + str(len(df_com_wsb_apple)))

# df_com_wsb_apple=df_com_wsb_apple.drop("link_id")
# df_wsb_apple=pl.concat([df_sub_wsb_apple,df_com_wsb_apple])

RES=np.array(text.str.contains('|'.join(['Amazon','AMAZON','AMZN','amzn'])))
RES2=np.array(text_2.str.contains('|'.join(['Amazon','AMAZON','AMZN','amzn'])))
df_sub_wsb_amazon=df_sub_wsb.filter(RES)
print('la lunghezza del dataframe di submissions filtrato è ' + str(len(df_sub_wsb_amazon)))
df_com_wsb_amazon=df_com_wsb.filter(RES2)
print('la lunghezza del dataframe di comments filtrato è ' + str(len(df_com_wsb_amazon)))
df_com_wsb_amazon=df_com_wsb_amazon.drop("link_id")
df_wsb_amazon=pl.concat([df_sub_wsb_amazon,df_com_wsb_amazon])

# RES=np.array(text.str.contains('|'.join(['Google','GOOGLE','GOOG','goog','Alphabet','ALPHABET'])))
# RES2=np.array(text_2.str.contains('|'.join(['Google','GOOGLE','GOOG','goog','Alphabet','ALPHABET'])))
# df_sub_wsb_apple=df_sub_wsb.filter(RES)
# print('la lunghezza del dataframe di submissions filtrato è ' + str(len(df_sub_wsb_apple)))
# df_com_wsb_apple=df_com_wsb.filter(RES2)
# print('la lunghezza del dataframe di comments filtrato è ' + str(len(df_com_wsb_apple)))
# df_com_wsb_apple=df_com_wsb_apple.drop("link_id")
# df_wsb_google=pl.concat([df_sub_wsb_apple,df_com_wsb_apple])

RES=np.array(text.str.contains('|'.join(['Google','GOOGLE','GOOG','goog'])))
RES2=np.array(text_2.str.contains('|'.join(['Google','GOOGLE','GOOG','goog'])))
df_sub_wsb_google=df_sub_wsb.filter(RES)
print('la lunghezza del dataframe di submissions filtrato è ' + str(len(df_sub_wsb_google)))
df_com_wsb_google=df_com_wsb.filter(RES2)
print('la lunghezza del dataframe di comments filtrato è ' + str(len(df_com_wsb_google)))
df_com_wsb_google=df_com_wsb_google.drop("link_id")
df_wsb_google=pl.concat([df_sub_wsb_google,df_com_wsb_google])

la lunghezza del dataframe di submissions filtrato è 1180
la lunghezza del dataframe di comments filtrato è 2541
la lunghezza del dataframe di submissions filtrato è 691
la lunghezza del dataframe di comments filtrato è 3341


In [21]:
# df_sub_wsb_amazon.write_ndjson(r'C:\Tempor\reddit\network\df_sub_wsb_amazon_5.json')
# df_sub_wsb_google.write_ndjson(r'C:\Tempor\reddit\network\df_sub_wsb_google_5.json')

In [18]:
# df_wsb_amazon=df_wsb_amazon.with_columns(pl.col('created_utc').str.to_datetime())
# df_wsb_google=df_wsb_google.with_columns(pl.col('created_utc').str.to_datetime())

In [17]:
# df_wsb_apple=df_wsb_apple.with_columns(pl.col("created_utc").dt.week().alias("week"))

df_wsb_amazon=df_wsb_amazon.with_columns(pl.col("created_utc").dt.week().alias("week"))
df_wsb_google=df_wsb_google.with_columns(pl.col("created_utc").dt.week().alias("week"))

In [18]:
# week_list=df_wsb_apple.partition_by(["week"])
week_list_amazon=df_wsb_amazon.partition_by(["week"])
week_list_google=df_wsb_google.partition_by(["week"])

In [19]:
# list=week_list[1]
# list

In [20]:
# list

In [21]:
# len(list)

In [22]:
# list.write_ndjson(r'C:\Tempor\reddit\sentiment_data\apple\2015_2_6.json')

In [23]:
# for lists in week_list:
#     lists.write_ndjson(r'C:\Tempor\reddit\sentiment_data\2015_10_'+str(lists['week'][0])+'.json')

for lists in week_list_amazon:
    lists.write_ndjson(r'C:\Tempor\reddit\sentiment_data\amazon\2016_4_'+str(lists['week'][0])+'.json')

for lists in week_list_google:
    lists.write_ndjson(r'C:\Tempor\reddit\sentiment_data\google\2016_4_'+str(lists['week'][0])+'.json')

Concatenate the same weeks that are in different months

In [115]:
a_week_list_set=pl.read_ndjson(r'C:\Tempor\reddit\sentiment_data\amazon\2016_5_22.json')
a_week_list_ott=pl.read_ndjson(r'C:\Tempor\reddit\sentiment_data\amazon\2016_6_22.json')

g_week_list_set=pl.read_ndjson(r'C:\Tempor\reddit\sentiment_data\google\2016_5_22.json')
g_week_list_ott=pl.read_ndjson(r'C:\Tempor\reddit\sentiment_data\google\2016_6_22.json')

In [116]:
len(a_week_list_ott)

600

In [117]:
a_week_list5=pl.concat([a_week_list_set,a_week_list_ott])

g_week_list5=pl.concat([g_week_list_set,g_week_list_ott])

In [118]:
len(g_week_list5)

1116

In [ ]:
# Now change the numbers *****************

In [119]:
a_week_list5.write_ndjson(r'C:\Tempor\reddit\sentiment_data\amazon\2016_6_22.json')

g_week_list5.write_ndjson(r'C:\Tempor\reddit\sentiment_data\google\2016_6_22.json')

In [120]:
prova=pl.read_ndjson(r'C:\Tempor\reddit\sentiment_data\google\2016_6_22.json')
len(prova)

1116